**这里的m支持随机**

In [1]:
import networkx as nx
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
import sys
from dgl.nn import GraphConv#instead of GCNConv in PyG
import dgl
sys.path.append('..')
from model.models_dgl import GraphConvWL
from ba4label_dgl_total_m import *
from torch import nn
def generate_single_sample(label, perturb_type, nodes_num = 25, m = 1, perturb_dic = {}, 
seed = None):
    '''
    return a networkx instance
    '''
    basis_type = "ba"
    which_type = label
    if which_type == 0:
        if perturb_type == 0:
            G, role_id, plug_id = build_graph(nodes_num, basis_type, [], start = 0, m = m, seed = seed)
        else:
            G, role_id, plug_id = build_graph(nodes_num - perturb_type, basis_type, [[perturb_dic[perturb_type]]], start = 0, m = m, seed = seed)
    else:
        list_shapes = [["house"]] * (which_type - 1) + [["five_cycle"]] * (3 - which_type)
        if perturb_type != 0:
            list_shapes = list_shapes + [[perturb_dic[perturb_type]]]
        G, role_id, plug_id = build_graph(nodes_num-10-perturb_type, basis_type, list_shapes, start = 0, m = m, seed = seed)
    return G

class FixedNet(nn.Module):
    def __init__(self, num_node_features, num_classes, num_layers, concat_features, conv_type):
        super(FixedNet, self).__init__()
        dim = 1
        self.convs = torch.nn.ModuleList()
        if conv_type == 'GraphConvWL':#'GCNConv':
            conv_class = GraphConvWL
            #kwargs = {'add_self_loops': False}
        elif conv_type == 'GraphConv':
            conv_class = GraphConv
            kwargs = {}
        else:
            raise RuntimeError(f"conv_type {conv_type} not supported")

        self.convs.append(conv_class(num_node_features, dim, bias = False))#, **kwargs))
        for i in range(num_layers - 1):
            self.convs.append(conv_class(dim, dim, bias = False))#, **kwargs))
        self.concat_features = concat_features


    def forward(self, g, x):
        '''
        g: DGL Graph
        x: node feature
        '''
        xs = [x]
        for conv in self.convs:
            x = conv(g, x)
            x = F.relu(x)
            xs.append(x)
        if self.concat_features:
            x = torch.cat(xs, dim=1)
        g.ndata['h'] = x
        hg = dgl.mean_nodes(g, 'h')
        return hg, xs

Using backend: pytorch


In [3]:
from tqdm import tqdm as tq
from dgl.nn import GraphConv
class GraphConvWL(nn.Module):
    r'''
    Description
    -----------
    Similar to GraphConv in PyG

    This graph convolution operater was introduced in `"Weisfeiler and Leman Go
    Neural: Higher-order Graph Neural Networks"
    <https://arxiv.org/abs/1810.02244>`_ paper

    Parameters
    ----------
    in_feats : int
        Input feature size; i.e, the number of dimensions of :math:`h_j^{(l)}`.
    out_feats : int
        Output feature size; i.e., the number of dimensions of :math:`h_i^{(l+1)}`.
    bias : bool, optional
        If True, apply a learnable bias to the output. Default: ``True``
    allow_zero_in_degree : bool, optional
        If there are 0-in-degree nodes in the graph, output for those nodes will be invalid
        since no message will be passed to those nodes. This is harmful for some applications
        causing silent performance regression. This module will raise a DGLError if it detects
        0-in-degree nodes in input graph. By setting ``True``, it will suppress the check
        and let the users handle it by themselves. Default: ``True``.
    '''
    def __init__(self,
                in_feats,
                out_feats,
                bias = True,
                allow_zero_in_degree = True) -> None:
        super().__init__()
        self.conv_from_neigh = GraphConv(in_feats, out_feats, norm = 'none', weight = True, bias = bias, allow_zero_in_degree = allow_zero_in_degree)
        self.conv_from_self = nn.Linear(in_feats,out_feats, bias = False)

        self.reset_parameters()
    
    def reset_parameters(self):
        self.conv_from_neigh.reset_parameters()
        self.conv_from_self.reset_parameters()

    def forward(self, graph, feat):
        neigh_feat = self.conv_from_neigh(graph, feat)
        self_feat = self.conv_from_self(feat)
        return neigh_feat+self_feat


class Net3(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, num_layers, concat_features, conv_type, dim = 1, report = False, readout = 'Mean'):
        super(Net3, self).__init__()
        dim = dim
        self.report = report
        self.readout = readout
        self.convs = torch.nn.ModuleList()
        if conv_type == 'GraphConvWL':#'GCNConv':
            conv_class = GraphConvWL
            #kwargs = {'add_self_loops': False}
        elif conv_type == 'GraphConv':
            conv_class = GraphConv
            kwargs = {}
        else:
            raise RuntimeError(f"conv_type {conv_type} not supported")

        self.convs.append(conv_class(num_node_features, dim))#, **kwargs))
        for i in range(num_layers - 1):
            self.convs.append(conv_class(dim, dim))#, **kwargs))
        self.concat_features = concat_features
        if concat_features:
            self.fc = Linear(dim * num_layers + num_node_features, num_classes)
        else:
            self.fc = Linear(dim, num_classes)

    def forward(self, g, x):
        '''
        g: DGL Graph
        x: node feature
        '''
        xs = [x]
        for conv in self.convs:
            x = conv(g, x)
            if self.report:
                print(x)
            x = F.relu(x)
            xs.append(x)
        if self.concat_features:
            x = torch.cat(xs, dim=1)
        g.ndata['h'] = x
        if self.readout == 'Mean':
            hg = dgl.mean_nodes(g, 'h')
        elif self.readout == 'Max':
            hg = dgl.max_nodes(g, 'h')
        if self.report:
            print(hg)
        hg = self.fc(hg)
        return F.log_softmax(hg, dim=1)
def model_train_with_dim(layers, dim, epochs_num = 150, m = 5, readout = 'Mean', graphs_num = 1000):
    
    EPOCHS_NUM = epochs_num
    m = m
    device = torch.device('cuda')
    data = BA4labelDataset(graphs_num = graphs_num, m = m,nodes_num=25, perturb_dic = {4:'square_diagonal'})
    dataloader = dgl.dataloading.GraphDataLoader(data, batch_size = 16, shuffle = True)
    data = BA4labelDataset(graphs_num = graphs_num, m = m,nodes_num=25, perturb_dic = {4:'square_diagonal'})
    testdataloader = dgl.dataloading.GraphDataLoader(data, batch_size = 16, shuffle = True)
    model = Net3(data.num_node_features, data.num_classes, layers, True,
                                        'GraphConvWL', dim = dim, readout = readout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0)

    model.train()
    pbar = tq(range(EPOCHS_NUM))
    for epoch in pbar:
        #train
        loss_all = 0
        for g, labels in dataloader:
            g = g.to(device)
            optimizer.zero_grad()
            output = model(g, g.ndata['x'])
            loss = F.nll_loss(output, labels.to(device))
            loss.backward()
            loss_all += loss.item()
            optimizer.step()
        train_loss = loss_all/len(dataloader)
        #train_acc
        model.eval()

        correct = 0
        total = 0
        for g, labels in dataloader:
            g = g.to(device)
            output = model(g, g.ndata['x'])
            pred = output.max(dim=1)[1]
            eq_pred = pred.eq(labels.to(device))
            correct += eq_pred.sum().item()
            total += len(labels.to(device))
        train_acc = correct/total
        #test_acc
        model.eval()

        correct = 0
        total = 0
        for g, labels in testdataloader:
            g = g.to(device)
            output = model(g, g.ndata['x'])
            pred = output.max(dim=1)[1]
            eq_pred = pred.eq(labels.to(device))
            correct += eq_pred.sum().item()
            total += len(labels.to(device))
        test_acc = correct/total

        pbar.set_postfix(train_loss=train_loss, train_acc = train_acc, test_acc = test_acc)
    return model, train_acc,test_acc

In [4]:
def test_model_with_another_m(model_test, m):
    model_test.eval()
    data = BA4labelDataset(m = m, nodes_num=25, perturb_dic = {4:'square_diagonal'})
    testdataloader = dgl.dataloading.GraphDataLoader(data, batch_size = 16, shuffle = True)
    correct = 0
    total = 0
    for g, labels in testdataloader:
        g = g.to(device)
        output = model_test(g, g.ndata['x'])
        pred = output.max(dim=1)[1]
        eq_pred = pred.eq(labels.to(device))
        correct += eq_pred.sum().item()
        total += len(labels.to(device))
    test_acc = correct/total
    return test_acc

In [5]:
device = torch.device('cuda')
result = []
for _ in range(10):
    model_5, train_acc_5, test_acc_5 = model_train_with_dim(3,32,150, m = None, readout = 'Max', graphs_num = 3000)
    temp = test_model_with_another_m(model_5, 2)
    print('Acc on dataset m = 2', temp)
    result.append([train_acc_5, test_acc_5,temp])
print(result, np.mean(result, axis = 0))

100%|██████████| 150/150 [04:49<00:00,  1.93s/it, test_acc=0.954, train_acc=0.967, train_loss=0.0526]


Acc on dataset m = 2 0.866


100%|██████████| 150/150 [04:52<00:00,  1.95s/it, test_acc=1, train_acc=1, train_loss=0.00142]        


Acc on dataset m = 2 0.948


100%|██████████| 150/150 [04:50<00:00,  1.94s/it, test_acc=0.989, train_acc=0.993, train_loss=0.0312]


...

Acc on dataset m = 2 0.992


100%|██████████| 150/150 [05:29<00:00,  2.20s/it, test_acc=0.997, train_acc=1, train_loss=0.000191]   


Acc on dataset m = 2 0.999


100%|██████████| 150/150 [05:29<00:00,  2.20s/it, test_acc=0.967, train_acc=0.976, train_loss=0.077] 


Acc on dataset m = 2 0.79
[[0.9673333333333334, 0.9543333333333334, 0.866], [1.0, 0.9996666666666667, 0.948], [0.9926666666666667, 0.9886666666666667, 0.84], [0.9903333333333333, 0.9886666666666667, 0.828], [0.99, 0.984, 0.799], [0.97, 0.9623333333333334, 0.751], [0.979, 0.9813333333333333, 0.751], [0.9996666666666667, 0.9986666666666667, 0.992], [1.0, 0.9973333333333333, 0.999], [0.9763333333333334, 0.967, 0.79]] [0.98653333 0.9822     0.8564    ]


In [6]:
import pandas as pd
result_df = pd.DataFrame(result, columns = ['train_acc','test_acc','test2_acc'])
result_df.to_csv('./result/sample3k_max_randomm.csv')

In [8]:
best_model = None
best_acc = 0
device = torch.device('cuda')
result = []
for _ in range(10):
    model_5, train_acc_5, test_acc_5 = model_train_with_dim(3,32,150, m = None, readout = 'Mean', graphs_num = 3000)
    temp = test_model_with_another_m(model_5, 2)
    print('Acc on dataset m = 2', temp)
    if temp > best_acc:
        best_model = model_5
        best_acc = temp
    result.append([train_acc_5, test_acc_5,temp])
print(result, np.mean(result, axis = 0))

100%|██████████| 150/150 [05:28<00:00,  2.19s/it, test_acc=0.999, train_acc=0.999, train_loss=0.00321]


Acc on dataset m = 2 0.742


100%|██████████| 150/150 [05:29<00:00,  2.20s/it, test_acc=0.999, train_acc=0.999, train_loss=0.0024] 


Acc on dataset m = 2 0.851


100%|██████████| 150/150 [05:28<00:00,  2.19s/it, test_acc=1, train_acc=1, train_loss=0.000735]       


...

Acc on dataset m = 2 0.743


100%|██████████| 150/150 [06:39<00:00,  2.66s/it, test_acc=1, train_acc=1, train_loss=0.000697]       


Acc on dataset m = 2 0.675


100%|██████████| 150/150 [06:36<00:00,  2.64s/it, test_acc=0.999, train_acc=0.999, train_loss=0.00896]


Acc on dataset m = 2 0.868
[[0.9993333333333333, 0.999, 0.742], [0.9993333333333333, 0.999, 0.851], [1.0, 0.9996666666666667, 0.736], [0.9996666666666667, 0.9983333333333333, 0.789], [0.9996666666666667, 0.9993333333333333, 0.734], [1.0, 0.9996666666666667, 0.881], [0.995, 0.9926666666666667, 0.545], [0.9996666666666667, 0.999, 0.743], [1.0, 1.0, 0.675], [0.9986666666666667, 0.999, 0.868]] [0.99913333 0.99856667 0.7564    ]
